# 1. Libraries

In [1]:
### Computational Libraries
import numpy as np
import pandas as pd

### HTML Servlet related
import requests
import markdown2
from xml.dom.minidom import parseString


### News Summary Library
import newspaper

### NLTK
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords

### String Manipulation
from collections import defaultdict
from string import punctuation
from heapq import nlargest

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

#### From Markdown to HTML

from webbrowser import open_new_tab


### Local HTML to PDF

import pdfcrowd
import sys

## Date for convention

from datetime import date
import calendar

# 2. Defined Classes

## 2.1 Class 'FrequencySummarizer' for Summarizing a Text

In [2]:
class FrequencySummarizer:
  def __init__(self, min_cut=0.5, max_cut=0.95):
    """ Words that have a frequency term lower than min_cut or higer than max_cut will be ignored. """
    self._min_cut = min_cut
    self._max_cut = max_cut 
    self._stopwords = set(stopwords.words('english') + list(punctuation))

  def _compute_frequencies(self, word_sent):
    """ 
      Input: a list of sentences already tokenized.
      Output:  a dictionary where freq[w] is the frequency of w.
    """
    freq = defaultdict(int)
    for s in word_sent:
      for word in s:
        if word not in self._stopwords:
          freq[word] += 1
    # frequencies normalization and fitering
    m = float(max(freq.values()))
    for w in list(freq):
      freq[w] = freq[w]/m
      if freq[w] >= self._max_cut or freq[w] <= self._min_cut:
        del freq[w]
    return freq

  def summarize(self, text, n):
    """"
    Return a list of n sentences which represent the summary of text.
    """
    sents = sent_tokenize(text)
    assert n <= len(sents)
    word_sent = [word_tokenize(s.lower()) for s in sents]
    self._freq = self._compute_frequencies(word_sent)
    ranking = defaultdict(int)
    for i,sent in enumerate(word_sent):
      for w in sent:
        if w in self._freq:
          ranking[i] += self._freq[w]
    sents_idx = self._rank(ranking, n)    
    return [sents[j] for j in sents_idx]

  def _rank(self, ranking, n):
    """ return the first n sentences with highest ranking """
    return nlargest(n, ranking, key=ranking.get)

# 3. Web Search

*  The Data will be collected from the following News Sources only via Google-News:
    - goal.com  [HomePage](https://www.goal.com/en)
    - skysports.com  [HomePage](https://www.skysports.com/football)
    - espn.in  [HomePage](http://www.espn.in/football/?src=com)
    - theguardian.com  [HomePage](https://www.theguardian.com/football)


*  The News window is set to past 7 days.

- This is not an exhaustive list. You can customize this list by adding/removing websites. You can also filter particluar websited that you don't want in your search list.

- News wrt **Liverpool** and **English Premier League** will be fetched.
- News wrt **Liverpool** and **Champions League** will also be fetched separately.
- Then, I will combine both the News sources to only look for unique News.

# 4. News Compilation

In [3]:
urls = 0    ####Initialisation of the urls variable

sites = ['goal.com', 'skysports.com', 'espn.in', 'theguardian.com/football']    #### List to store the Websites

## 4.1 Function to fetch the URL and Title.

In [4]:
def URL_Decorator( func):
    """
    *  The function searches for the news as entered in the term 'search_string'
    *  The function also returns the title of the News for the fetched URL - retured by 'return_title' """
    def wrapper(string, days, urls):
        """ A Function Wrapper """
        term, count = func(string,days, urls)
        results = []
        results_title = []
        results_url = []
        obj = parseString( requests.get('https://news.google.com/rss/search?q={term}&hl=en-IN&gl=IN&ceid=IN:en'.format(term=term)).text )
        items = obj.getElementsByTagName('item') # Get each item
        for item in items[:count]:
            t,l = '', ''
            for node in item.childNodes:
                if node.nodeName == 'title':
                    t = node.childNodes[0].data
                elif node.nodeName == 'link':
                    l = node.childNodes[0].data
            results.append( (t,l) )
            results_title.append(t)
            results_url.append(l)
        return results_title, results_url
        
    return wrapper
    

@URL_Decorator
def search_strings(search_string, days, url):
    """
    A Function to write automated script as an input for 'get_google_new_results_test' function.
    
    search_string = Value that we need to search
    days = No of Days in the past
    urls = No of URL's to fetch
    """
    urls = int(url)
    for site in sites:
        if site == sites[0]:
            string = str('{search_string} {site}').format(search_string=search_string, site=site)
        elif site != sites[-1]:
            string = str(string) + str(' OR ') + str('{site}').format(site= site)
        else:
            string = string + str(' when:{day}d').format(day = days)
    return string, urls

# 7. Scraping Data into Markdown File

In [5]:
## Defining the Class Object ##
fs = FrequencySummarizer()

In [5]:
### Adding Naming Convension - based on Date
today_date = date.today()
today_date

day_file_name = str(today_date.day)+'/'+today_date.strftime('%m')+'/'+str(today_date.year)   ## In case Publish Date is absent
file2 = str(today_date.day)+'_'+today_date.strftime('%m')+'_'+str(today_date.year)
file_name = file2 + '_Report.pdf'

final_file_name = str(file_name)

In [111]:
%%capture output

### Setting search term
search_term = ['Liverpool AND English Premier League', 'Liverpool AND Champions League']
headings = ['English Premier League', 'Champions League']    #### Setting Headings

total_urls = []

for term, heading in zip(search_term,headings):
    
    title, urls = search_strings(('{0}').format(term), 20, 3)
    urls =  list(set(urls)^set(list(set(urls).intersection(total_urls))))  ## Only Unique URL's with respect to that fruit
    total_urls.extend(urls)
    
    print('<center><h3>' + str('{heading}  Overview').format(heading = str(heading))+'</h3></center>')

    for url in urls:
        try:
            common_stop_words = stopwords.words('english')  # Variable to store common English Stop words
            article = newspaper.Article(url)
            article.download()
            article.parse()
            article.nlp()

            title_url = article.title   # News Title
            text_text = article.text    # News Text
            title_genre =  article.keywords[0:5]
            #words = [ss('english').stem(word) for word in title_genre if word not in common_stop_words]
            words = [word for word in title_genre if word not in common_stop_words]

            summary = sent_tokenize(article.summary)
                
        except:
            continue

        ####################### Publish Date ####################################
        
        try:
            if(int(article.publish_date.month)>10):

                publish_date = str(article.publish_date.day)+'/' + str(article.publish_date.month)+'/' + str(article.publish_date.year)
                publish_date = str(publish_date)
            else:
                publish_date = str(article.publish_date.day)+'/0' + str(article.publish_date.month)+'/' + str(article.publish_date.year)
                publish_date = str(publish_date)
        except AttributeError:
            publish_date = str(day_file_name)
        ####################### Publish Date Ends ###############################


        ######################### Title #########################################
        print('<table>')

        print('<tr>')
        print('<th>')

        
        print('<a style= "color:#333399;font-family: "Times New Roman", Times, serif;font-size:110%;" href = "{url} ">'.format(url = url)+ str(title_url) +'</a>')
        

        print('</th>')
        print('</tr>')
        ######################### Share Widgets begins ######################################
        
       
        print('<a href="mailto:?subject={subject_title}&amp;body= {article_url} " title="Share by Email"> <img src="https://github.com/S-B-Iqbal/Test/raw/master/Images/outlook.png" alt="Outlook" style="float:right;height:20px;width:20px" hspace = "20"> </a>'.format(subject_title = title_url, article_url = str(url)))
        
        print('<a href="https://wa.me/send?text ={share_url}" ><img src="https://github.com/S-B-Iqbal/Test/raw/master/Images/whatsapp.png" alt="Whatsapp" style="float:right; height:20px;width:20px" hspace = "40"></a>'.format(share_url = str(url)))
        
        ######################### Share Widgets Ends ###################################### 
        
        
        print('</table>')              
        
        #print('    ')
        #print('    ')
        print('<article><strong>'+format(publish_date) + '</article>')
        
        ######################### Title Ends ######################################
        
        #print('\n {0}'.format(title_url))
        
        
        try:
            print(' ')
            print('<article>' + '<b style= "color:grey">Keywords:</b> <i>' + ' | '.join(words) + '</i> </article>')
            print('    ')
            print('------------------------------------------------------------------')
            """ Printing the Summary as bullet points."""
            print('<ul style="color:#000000; font-family:calibri">')
            for s in summary:
                print('*  '+ str(s))
            print('    ')
            
            print('    ')
            print('</ul>')
            print('------------------------------------------------------------------')
            print('    ')
            
            print('------------------------------------------------------------------')
            print('    ')
            print('    ')
            
        except AssertionError:
            continue
            
del total_urls

In [112]:
### Getting Text of the Output
Summary = output.stdout

In [113]:
## Direct Markdown conversion of Summary in 'test_summary'
test_summary = Markdown(data= Summary)
test_summary

<center><h3>English Premier League  Overview</h3></center>
<table>
<tr>
<th>
<a style= "color:#333399;font-family: "Times New Roman", Times, serif;font-size:110%;" href = "https://www.goal.com/en/news/sturridge-offered-to-several-a-league-clubs/adodhuf0hnkg1a658ll2gj3bb ">Liverpool striker Daniel Sturridge reportedly offered to 'several' A-League clubs</a>
</th>
</tr>
<a href="mailto:?subject=Liverpool striker Daniel Sturridge reportedly offered to 'several' A-League clubs&amp;body= https://www.goal.com/en/news/sturridge-offered-to-several-a-league-clubs/adodhuf0hnkg1a658ll2gj3bb " title="Share by Email"> <img src="https://github.com/S-B-Iqbal/Test/raw/master/Images/outlook.png" alt="Outlook" style="float:right;height:20px;width:20px" hspace = "20"> </a>
<a href="https://wa.me/send?text =https://www.goal.com/en/news/sturridge-offered-to-several-a-league-clubs/adodhuf0hnkg1a658ll2gj3bb" ><img src="https://github.com/S-B-Iqbal/Test/raw/master/Images/whatsapp.png" alt="Whatsapp" style="float:right; height:20px;width:20px" hspace = "40"></a>
</table>
<article><strong>25/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>liverpool | aleague | come | daniel | sturridge</i> </article>
    
------------------------------------------------------------------
<ul style="color:#000000; font-family:calibri">
*  Sturridge offered to 'several' A-League clubsThe English striker could be heading Down UnderCurrent player Daniel Sturridge has emerged as a surprise marquee candidate for the A-League with Brisbane Roar shaping as his most likely destination.
*  "One name I will throw up, and I know this player has already been offered to several A-League clubs already, is Daniel Sturridge," Shaw said on the Fox Football Podcast .
*  "I wouldn’t be at all surprised if the phone call is made from Robbie Fowler to bring someone like Daniel Sturridge down as a marquee next season."
*  While now at Liverpool, Sturridge began his career with City before moving to where he won the Premier League in 2010 and in 2012.
*  Sturridge spent some of last season on loan with and is off contract with Liverpool at season's end.
    
    
</ul>
------------------------------------------------------------------
    
------------------------------------------------------------------
    
    
<table>
<tr>
<th>
<a style= "color:#333399;font-family: "Times New Roman", Times, serif;font-size:110%;" href = "https://www.goal.com/en-in/news/premier-league-should-do-more-to-help-english-teams-in/1qcmwazazud6e1jui73hndvajm ">Liverpool News: 'Premier League should do more to help English clubs in Europe' - Klopp</a>
</th>
</tr>
<a href="mailto:?subject=Liverpool News: 'Premier League should do more to help English clubs in Europe' - Klopp&amp;body= https://www.goal.com/en-in/news/premier-league-should-do-more-to-help-english-teams-in/1qcmwazazud6e1jui73hndvajm " title="Share by Email"> <img src="https://github.com/S-B-Iqbal/Test/raw/master/Images/outlook.png" alt="Outlook" style="float:right;height:20px;width:20px" hspace = "20"> </a>
<a href="https://wa.me/send?text =https://www.goal.com/en-in/news/premier-league-should-do-more-to-help-english-teams-in/1qcmwazazud6e1jui73hndvajm" ><img src="https://github.com/S-B-Iqbal/Test/raw/master/Images/whatsapp.png" alt="Whatsapp" style="float:right; height:20px;width:20px" hspace = "40"></a>
</table>
<article><strong>20/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>dont | liverpool | premier | klopp | europe</i> </article>
    
------------------------------------------------------------------
<ul style="color:#000000; font-family:calibri">
*  Premier League should do more to help English teams in Europe - KloppThe Reds boss believes top-flight could do more to aid their clubs on the continent, as the Dutch authorities have done for Ajaxmanager Jurgen Klopp has called on the Premier League to help English teams in European competitions.
*  In contrast, Liverpool must play vital games in the Premier League run in before their own semi-final with .
*  “The Dutch league had a problem because for the last two match days, everyone plays at the same time,” Klopp told a press conference.
*  No, I don’t think it would happen here or in , to be 100 per cent honest.
*  Klopp said that the television deal for the Premier League made it difficult for the governing body to move the schedule as they have in the .
    
    
</ul>
------------------------------------------------------------------
    
------------------------------------------------------------------
    
    
<table>
<tr>
<th>
<a style= "color:#333399;font-family: "Times New Roman", Times, serif;font-size:110%;" href = "https://www.goal.com/en-us/news/why-most-man-utd-fans-want-man-city-to-win-premier-league/181ycus3aw0gj1mhvgsa5fir7h ">Why do Man Utd fans want Man City to win Premier League instead of Liverpool?</a>
</th>
</tr>
<a href="mailto:?subject=Why do Man Utd fans want Man City to win Premier League instead of Liverpool?&amp;body= https://www.goal.com/en-us/news/why-most-man-utd-fans-want-man-city-to-win-premier-league/181ycus3aw0gj1mhvgsa5fir7h " title="Share by Email"> <img src="https://github.com/S-B-Iqbal/Test/raw/master/Images/outlook.png" alt="Outlook" style="float:right;height:20px;width:20px" hspace = "20"> </a>
<a href="https://wa.me/send?text =https://www.goal.com/en-us/news/why-most-man-utd-fans-want-man-city-to-win-premier-league/181ycus3aw0gj1mhvgsa5fir7h" ><img src="https://github.com/S-B-Iqbal/Test/raw/master/Images/whatsapp.png" alt="Whatsapp" style="float:right; height:20px;width:20px" hspace = "40"></a>
</table>
<article><strong>24/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>united | liverpool | majority | win | utd</i> </article>
    
------------------------------------------------------------------
<ul style="color:#000000; font-family:calibri">
*  Wind forward to 2019 and Liverpool are attempting to finally win a 19th league title 29 years and 13 United crowns on from their last.
*  Article continues belowFor United fans, this might be the worst-case scenario.
*  They have no hope of a title themselves, and it is City and Liverpool who are battling for the crown.
*  “And whilst that might change in the future, at the moment the majority of United fans I’d say would consider Liverpool to be our biggest rivals.
*  But, whatever the result at Old Trafford, don’t be too surprised if you see United fans siding with City in the remainder of the run-in.
    
    
</ul>
------------------------------------------------------------------
    
------------------------------------------------------------------
    
    
<center><h3>Champions League  Overview</h3></center>
<table>
<tr>
<th>
<a style= "color:#333399;font-family: "Times New Roman", Times, serif;font-size:110%;" href = "https://www.goal.com/en/news/vidal-stakes-claim-for-liverpool-spot-as-barca-turn-focus/1ele7763c56611gsv0lqgs77x8 ">Vidal stakes claim for Liverpool spot as Barca turn focus towards Champions League</a>
</th>
</tr>
<a href="mailto:?subject=Vidal stakes claim for Liverpool spot as Barca turn focus towards Champions League&amp;body= https://www.goal.com/en/news/vidal-stakes-claim-for-liverpool-spot-as-barca-turn-focus/1ele7763c56611gsv0lqgs77x8 " title="Share by Email"> <img src="https://github.com/S-B-Iqbal/Test/raw/master/Images/outlook.png" alt="Outlook" style="float:right;height:20px;width:20px" hspace = "20"> </a>
<a href="https://wa.me/send?text =https://www.goal.com/en/news/vidal-stakes-claim-for-liverpool-spot-as-barca-turn-focus/1ele7763c56611gsv0lqgs77x8" ><img src="https://github.com/S-B-Iqbal/Test/raw/master/Images/whatsapp.png" alt="Whatsapp" style="float:right; height:20px;width:20px" hspace = "40"></a>
</table>
<article><strong>23/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>focus | best | stakes | liverpool | alaves</i> </article>
    
------------------------------------------------------------------
<ul style="color:#000000; font-family:calibri">
*  Vidal stakes claim for Liverpool spot as Barca turn focus towards Champions LeagueThe Chilean could join the lineup as the club edges closer int heir pursuit of a trebleearned a comfortable 2-0 win over on Tuesday which means they might be Spanish champions before they even play their next match, at on Saturday.
*  Th e treble is Barcelona’s dream and having won it in 2009 and 2015, hopes and expectations are rising.
*  Perhap s the player outside what is considered Valverde’s best XI with the best chance of breaking into the side is Arturo Vidal .
*  Liverpool’ s workhorses might look to strangle Arthur, Barcelona’s passmaster , out of the game, but Vidal would offer some means of protection .
*  Th e game was done and dusted by that point, with Luis Suarez slamming home from the penalty spot .
    
    
</ul>
------------------------------------------------------------------
    
------------------------------------------------------------------
    
    
<table>
<tr>
<th>
<a style= "color:#333399;font-family: "Times New Roman", Times, serif;font-size:110%;" href = "https://www.goal.com/en-bh/news/valverde-wary-of-powerful-liverpool-ahead-of-champions/ghz9od8lyiel15vz6yijas5en ">Barcelona news: Valverde wary of 'powerful' Liverpool ahead of Champions League semis</a>
</th>
</tr>
<a href="mailto:?subject=Barcelona news: Valverde wary of 'powerful' Liverpool ahead of Champions League semis&amp;body= https://www.goal.com/en-bh/news/valverde-wary-of-powerful-liverpool-ahead-of-champions/ghz9od8lyiel15vz6yijas5en " title="Share by Email"> <img src="https://github.com/S-B-Iqbal/Test/raw/master/Images/outlook.png" alt="Outlook" style="float:right;height:20px;width:20px" hspace = "20"> </a>
<a href="https://wa.me/send?text =https://www.goal.com/en-bh/news/valverde-wary-of-powerful-liverpool-ahead-of-champions/ghz9od8lyiel15vz6yijas5en" ><img src="https://github.com/S-B-Iqbal/Test/raw/master/Images/whatsapp.png" alt="Whatsapp" style="float:right; height:20px;width:20px" hspace = "40"></a>
</table>
<article><strong>20/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>liverpool | barcelona | premier</i> </article>
    
------------------------------------------------------------------
<ul style="color:#000000; font-family:calibri">
*  Valverde wary of 'powerful' Liverpool ahead of Champions League semisBarcelona head coach Ernesto Valverde expects a tough but attractive Champions League semi-final against the Premier League outfitErnesto Valverde has said he is wary of the "very powerful" frontline ahead of 's semi-final against the Premier League giants.
*  Barca are in control of ’s title race, while Liverpool are fighting for the Premier League crown and were runners-up in Europe's top club competition last season.
*  They are also fighting in the Premier League.
*  Between Barca and Liverpool, the two European powerhouses have won 10 Champions Leagues/European Cups, with only and winning the jug-eared trophy on more occasions.
*  Overall, the sides have met four times in the Champions League, with Liverpool eliminating the Catalan side over two legs in their last encounter, a last-16 tie in 2007.
    
    
</ul>
------------------------------------------------------------------
    
------------------------------------------------------------------
    
    
<table>
<tr>
<th>
<a style= "color:#333399;font-family: "Times New Roman", Times, serif;font-size:110%;" href = "https://www.goal.com/en-us/news/champions-league-semi-finals-fixtures-live-streams-how-to/1so7lxwo5fcws11wxzdwupxied ">Champions League semi-finals: Fixtures, live streams & how to watch</a>
</th>
</tr>
<a href="mailto:?subject=Champions League semi-finals: Fixtures, live streams & how to watch&amp;body= https://www.goal.com/en-us/news/champions-league-semi-finals-fixtures-live-streams-how-to/1so7lxwo5fcws11wxzdwupxied " title="Share by Email"> <img src="https://github.com/S-B-Iqbal/Test/raw/master/Images/outlook.png" alt="Outlook" style="float:right;height:20px;width:20px" hspace = "20"> </a>
<a href="https://wa.me/send?text =https://www.goal.com/en-us/news/champions-league-semi-finals-fixtures-live-streams-how-to/1so7lxwo5fcws11wxzdwupxied" ><img src="https://github.com/S-B-Iqbal/Test/raw/master/Images/whatsapp.png" alt="Whatsapp" style="float:right; height:20px;width:20px" hspace = "40"></a>
</table>
<article><strong>22/04/2019</article>
 
<article><b style= "color:grey">Keywords:</b> <i>watch | streams | live | champions | sport</i> </article>
    
------------------------------------------------------------------
<ul style="color:#000000; font-family:calibri">
*  Champions League semi-finals: Fixtures, live streams & how to watchEverything you need to know about the Champions League as it gets set to whittle the number of teams down to two, including how to watch every gameThe quest for continental glory is heating up with the moving into the semi-final phase.
*  Champions League semi-finals fixtures & resultsThis season's Champions League semi-finals will kick off on April 30, 2019 when Tottenham host Ajax at home and May 1, 2019 when Liverpool travel to Barcelona.
*  TV Channel, Live Stream & How To WatchIn the UK, Champions League semi-final matches will be available to watch live on TV via BT Sport , who also offer an online streaming service through their app.
*  In the US, viewers are able to watch the Champions League semi-final games on TV through the Univision network (which includes Univision Deportes, UniMas and Galavision) or TNT USA .
*  Semi-final One: Ajax vs TottenhamAjax vs Tottenham Semi-final Two: Barcelona vs LiverpoolWinners of the semi-finals will contest the 2019 Champions League final on June 1, 2019 .
    
    
</ul>
------------------------------------------------------------------
    
------------------------------------------------------------------
    
    


### Conversion of Markdown to HTML

In [114]:
html = markdown2.markdown(str(Summary), html4tags=True, tab_width=5,  )

### Cleaning the HTML File

In [115]:
### Rectify Out of Bound Paras
html = html.replace("<p>","<ul><li><p>")

html = html.replace('</p>', '</p></li></ul>')

#### Rectify Headlines Link
html = html.replace("<ul><li><p><a href","<p><a href")

html = html.replace('/a></p></li></ul>', '/a></p>')

###  Rectify <centre>
html = html.replace("<ul><li><p><center></p></li></ul>","<p><center></p>")

html = html.replace('<ul><li><p></center></p></li></ul>', '<p></center></p>')


### Rectify *

html = html.replace("*","</li><li>")

### Rectify Image

html = html.replace("<li><img ","<lily imagine ")

#html = html.replace("</h1></center></p></li></ul>","</h1></center></p>")

## Rectify Title Bullet point

html = html.replace("<ul><li><p><center><h3>","<p><center><h3>")

html = html.replace("</h3></center></p></li></ul>","</h3></center></p>")

## Open in new tab

html = html.replace("<a href", '<a rel="noopener noreferrer" target="_blank" href')



### Checking the sample Webpage

In [116]:
filename = 'File.html'

f = open(filename,'w', encoding='utf-16')

wrapper = """<!DOCTYPE html>
<html lang="en">



<head>

<style>
.center {
  text-align: center;
  
}




</style>


<meta charset="ISO-8859-1" >
<div class="page center">
   <a href="https://www.liverpoolfc.com/welcome-to-liverpool-fc"><img class="auto-size__target postpone-load--fade-in widge-figure__image" data-src="https://e1.365dm.com/18/06/768x432/skysports-liverpool-fixtures_4325753.jpg?20180601143103" alt="" itemprop="image" src="https://e1.365dm.com/18/06/768x432/skysports-liverpool-fixtures_4325753.jpg?20180601143103" style="PADDING-RIGHT: 70px"></a>
</div>

<title>%s</title>

</head>

<body>
    
    %s
</body>



</html>"""

title = str('Liverpool News')

whole = wrapper % (title, html)

#whole = wrapper % (html)

f.write(whole)

f.close()

open_new_tab(filename)

True

## Conversion to PDF

In [66]:
try:
    # create the API client instance
    client = pdfcrowd.HtmlToPdfClient('<user_name>', '<API_Key>')

    # run the conversion and write the result to a file
    client.convertFileToFile('File.html', final_file_name )
except pdfcrowd.Error as why:
    # report the error
    sys.stderr.write('Pdfcrowd Error: {}\n'.format(why))

    # handle the exception here or rethrow and handle it at a higher level
    raise